In [36]:
#import basic Libraries
import pandas as pd
import numpy as np
import numpy_indexed as npi

#Visualizaiton imports
import seaborn as sns
import matplotlib.pyplot as plt

#combine sparse matrices
import scipy.sparse as sps

#Importing Sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import RandomizedLogisticRegression
from sklearn.decomposition import MiniBatchDictionaryLearning

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#Webscraping (for locations)
#from bs4 import BeautifulSoup 
#import requests

from collections import defaultdict

%matplotlib inline

In [2]:
filenames = ['aisles','departments' ,'order_products__prior','order_products__train', 'orders','products']

In [3]:
#import of all files from the data folder.
aisles = pd.read_csv('./data/'+ filenames[0] + '.csv')
departments = pd.read_csv('./data/'+ filenames[1] + '.csv')
order_products__prior = pd.read_csv('./data/'+ filenames[2] + '.csv')
order_products__train = pd.read_csv('./data/'+ filenames[3] + '.csv')
orders = pd.read_csv('./data/'+ filenames[4] + '.csv')
products = pd.read_csv('./data/'+ filenames[5] + '.csv')

In [4]:
#checking the shape of each of the file and checks for any null values.
for name in filenames:
    print(name, eval(name).shape,": Null values?", eval(name).isnull().any()[0])

aisles (134, 2) : Null values? False
departments (21, 2) : Null values? False
order_products__prior (32434489, 4) : Null values? False
order_products__train (1384617, 4) : Null values? False
orders (3421083, 7) : Null values? False
products (49688, 4) : Null values? False


In [5]:
aisles.head()
departments.head()
#combine aisle and departments
order_products__prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [6]:
opt = order_products__train.merge(products,how='left', on='product_id')
opt = opt.merge(departments,how='left', on='department_id')
opt = opt.merge(aisles,how='left', on='aisle_id')

In [7]:
opt.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,1,49302,1,1,Bulgarian Yogurt,120,16,dairy eggs,yogurt
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses
2,1,10246,3,0,Organic Celery Hearts,83,4,produce,fresh vegetables
3,1,49683,4,0,Cucumber Kirby,83,4,produce,fresh vegetables
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,canned meat seafood


In [8]:
#instanciate Oone hot encoder ot break out the data my aisle.
ohe = OneHotEncoder()

In [9]:
#test to  see # of aisles
print('# of unique aisles =',len(opt['aisle_id'].unique()))
aisle = ohe.fit_transform(opt['aisle_id'].values.reshape(-1,1))

# of unique aisles = 134


In [10]:
#one hot encode the aisles and create a dense matrix.
a_ohe = pd.DataFrame(aisle.toarray())
aisle = a_ohe.to_dense()

In [11]:
#add order_id to the one hot encoded matrix
basket = opt[['order_id']].join(aisle)

In [12]:
basket.head()

,order_id,0,1,2,3,4,5,6,7,8,...,124,125,126,127,128,129,130,131,132,133
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#return the total items in the basket from the aisles
basket = basket.groupby('order_id').sum()

In [14]:
#covert any values to be a binary. 
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [15]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.15, use_colnames=True)

In [16]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(122),(23),0.383030,0.550099,0.287282,0.750025,1.363435,0.076577,1.799781
1,(23),(122),0.550099,0.383030,0.287282,0.522237,1.363435,0.076577,1.291371
2,(83),(23),0.223750,0.550099,0.151948,0.679099,1.234503,0.028864,1.401993
3,(23),(83),0.550099,0.223750,0.151948,0.276220,1.234503,0.028864,1.072494
4,"(82, 122)",(23),0.252361,0.550099,0.203919,0.808045,1.468908,0.065095,2.343786


In the above example, the {Diaper} is the **antecedent** and the {Beer} is the consequent. Both antecedents and consequents can have multiple items. In other words, {Diaper, Gum} -> {Beer, Chips} is a valid rule.

**Support** is the relative frequency that the rules show up. In many instances, you may want to look for high support in order to make sure it is a useful relationship. However, there may be instances where a low support is useful if you are trying to find “hidden” relationships.

**Confidence** is a measure of the reliability of the rule. A confidence of .5 in the above example would mean that in 50% of the cases where Diaper and Gum were purchased, the purchase also included Beer and Chips. For product recommendation, a 50% confidence may be perfectly acceptable but in a medical situation, this level may not be high enough.

**Lift** is the ratio of the observed support to that expected if the two rules were independent (see wikipedia). The basic rule of thumb is that a lift value close to 1 means the rules were completely independent. Lift values > 1 are generally more “interesting” and could be indicative of a useful rule pattern.

In [17]:
#sorts to see the lift greater than or equal to 1
rules[rules['lift'] >= 1].head()

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(122),(23),0.383030,0.550099,0.287282,0.750025,1.363435,0.076577,1.799781
1,(23),(122),0.550099,0.383030,0.287282,0.522237,1.363435,0.076577,1.291371
2,(83),(23),0.223750,0.550099,0.151948,0.679099,1.234503,0.028864,1.401993
3,(23),(83),0.550099,0.223750,0.151948,0.276220,1.234503,0.028864,1.072494
4,"(82, 122)",(23),0.252361,0.550099,0.203919,0.808045,1.468908,0.065095,2.343786


In [18]:
#test to  see # of aisles
print('# of unique product =',len(opt['product_id'].unique()))
print('# of unique order id =',len(opt['order_id'].unique()))

# of unique product = 39123
# of unique order id = 131209


In [19]:
#tranposes the data to ensure that it is correctly fitted.
product = ohe.fit_transform(opt['product_id'].values.reshape(-1,1))

In [20]:
#create a dense matrix. 
product = sps.csr_matrix(product,dtype='int64')

In [21]:
product

<1384617x39123 sparse matrix of type '<class 'numpy.int64'>'
	with 1384617 stored elements in Compressed Sparse Row format>

In [22]:
#turns orders into a condensed row to be combined with products
orders = sps.csr_matrix(opt['order_id'][:,None],dtype='int64')

In [23]:
print(orders.shape)

(1384617, 1)


In [24]:
#re joins the orders to the OHE products.
basket = sps.hstack((orders,product),dtype=('int64'))

In [25]:
#groupby data
basket.getcol(0).toarray()

array([[      1],
       [      1],
       [      1],
       ...,
       [3421070],
       [3421070],
       [3421070]], dtype=int64)

In [26]:
basket.getcol(0)

<1384617x1 sparse matrix of type '<class 'numpy.int64'>'
	with 1384617 stored elements in Compressed Sparse Row format>

In [48]:
basket_orders = npi.group_by(basket.getcol(0).toarray(), reduction='sum')

## stuck here

In [59]:
basket_orders

In [32]:
#this converts the sums to binary.
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = bask_ar.applymap(encode_units)

NameError: name 'bask_ar' is not defined

In [ ]:
frequent_itemsets = apriori(bask_ar, min_support=0.07, use_colnames=True)

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

## Importing locations for their website
url= 'https://www.instacart.com/locations'
response = requests.get(url, headers = {'User-agent': 'agentP'})
soup = BeautifulSoup(response.text,'lxml')

#finds the links to each individual state and city
href = soup.find_all(href=True)

Puts all the links into Dataframe.
total_locations = []
for i in range(len(href)):
    total_locations.append(href[i].get_text())

locations = total_locations[19:-21]

states = []
for i in locations:
    states.append(i.split('\n')[1])
location = pd.DataFrame({'state': states})

gets the states and the cities that they are located in.
states = location[:44]
locations = location[44:]

#renames the column to city
locations.columns=['city']

#resets the index to align it correctly.
locations.reset_index(inplace = True)

## Exports to CSV

#states.to_csv('./data/states.csv')
#locations.to_csv('./data/cities.csv')




In [ ]:
pd.options.display.max_columns = 40000